In [1]:

import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import re
import nltk
import torch

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.sentiment import SentimentIntensityAnalyzer
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax

plt.style.use('ggplot')
print("SUKSES")


SUKSES


In [12]:
import os
os.listdir("/kaggle/input/twittertoxictweets")

df = pd.read_csv("/kaggle/input/twittertoxictweets/twitter_toxic_tweets.csv")
df = df.head(500)   
df.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [15]:
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"http\S+|www\S+", "", text)
    text = re.sub(r"@\w+|#\w+", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

In [17]:
sia = SentimentIntensityAnalyzer()

def vader_sentiment(text):
    score = sia.polarity_scores(text)['compound']
    if score >= 0.05:
        return "positive"
    elif score <= -0.05:
        return "negative"
    else:
        return "neutral"

In [ ]:
MODEL = "cardiffnlp/twitter-roberta-base-sentiment"

tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

labels = ["negative", "neutral", "positive"]

def roberta_sentiment(text):
    encoded = tokenizer(text, return_tensors="pt", truncation=True)
    with torch.no_grad():
        output = model(**encoded)
    scores = output.logits[0].numpy()
    probs = softmax(scores)
    return labels[np.argmax(probs)]

'(MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /cardiffnlp/twitter-roberta-base-sentiment/resolve/main/tokenizer_config.json (Caused by NameResolutionError("HTTPSConnection(host=\'huggingface.co\', port=443): Failed to resolve \'huggingface.co\' ([Errno -3] Temporary failure in name resolution)"))'), '(Request ID: 1a2c2008-f321-479b-a3e8-f63cf6f0be9d)')' thrown while requesting HEAD https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment/resolve/main/tokenizer_config.json
Retrying in 1s [Retry 1/5].
'(MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /cardiffnlp/twitter-roberta-base-sentiment/resolve/main/tokenizer_config.json (Caused by NameResolutionError("HTTPSConnection(host=\'huggingface.co\', port=443): Failed to resolve \'huggingface.co\' ([Errno -3] Temporary failure in name resolution)"))'), '(Request ID: d1759fdf-6472-4204-8b6b-271ccde43cba)')' thrown while re

In [ ]:
df["clean_text"] = df["tweet"].astype(str).apply(preprocess_text)

df["vader_sentiment"] = df["tweet"].astype(str).apply(vader_sentiment)
df["roberta_sentiment"] = df["tweet"].astype(str).apply(roberta_sentiment)

df.head()

In [ ]:
plt.figure()
sns.countplot(x="vader_sentiment", data=df)
plt.title("VADER Sentiment Distribution")
plt.show()

plt.figure()
sns.countplot(x="roberta_sentiment", data=df)
plt.title("RoBERTa Sentiment Distribution")
plt.show()